In [1]:
#importing libraries

import matplotlib.pyplot as plt
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50

import pyspark
import tensorframes
import pyspark.sql.functions as f
import sparkdl as dl
from pyspark import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType, ArrayType
from keras.datasets import fashion_mnist
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf

2023-10-03 20:58:34.765911: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-03 20:58:34.766009: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-03 20:58:34.771291: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-03 20:58:35.389132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 20:58:39.668796: W tensorflow/compiler/tf2t

In [2]:
#initalizing spark session
spark = SparkSession.builder \
.appName("sba22177_Integrated_CA01_2") \
.getOrCreate()

2023-10-02 22:23:55,564 WARN util.Utils: Your hostname, fiona-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2023-10-02 22:23:55,567 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/spark-3.2.4-bin-hadoop3.2/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hduser/.ivy2/cache
The jars for the packages stored in: /home/hduser/.ivy2/jars
databricks#spark-deep-learning added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e58085c-31ea-493c-902a-35085abcfaa0;1.0
	confs: [default]
:: resolution report :: resolve 4017ms :: artifacts dl 1ms
	:: modules in use:
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   0   |   0   |
	---------------------------------------------------------------------

:: problems summary ::
:::: WARNINGS
		module not found: databricks#spark-deep-learning;0.1.0-spark2.1-s_2.11.

	==== local-m2-cache: tried

	  file:/home/hduser/.m2/repository/databricks/spark-deep-learn

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
#dowload the MNIST data
(train_X, train_y), (test_X, test_y) = fashion_mnist.load_data()

In [ ]:
#examining dataset
print(test_X.shape)
print(train_X.shape)
print(test_y.shape)
print(train_y.shape)

In [ ]:
#viewing unique labels
set(train_y)

In [ ]:
#defining the image classes so not just numbers as above
class_names = ['T-shirt/top','Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
#feature scaling colours from 0-255 to 0-1
train_X = train_X/255.0
test_X = test_X/255.0

In [ ]:
#Showing the first 10 images
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_X[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_y[i]])
plt.show()

In [ ]:
train_X_df = train_X.reshape((train_X.shape[0], -1))
test_X_df = test_X.reshape((test_X.shape[0], -1))

In [ ]:
#examining dataset
print(test_X_df.shape)
print(train_X_df.shape)

In [ ]:
train_code=1
test_code=2

In [ ]:
train_data=[
    (i,
    train_X_df[i].astype(int).tolist(),
    int(train_y[i]),
    train_code,
    )  for i in range(len(train_y))
]

In [ ]:
test_data=[
    (i,
    test_X_df[i].astype(int).tolist(),
    int(test_y[i]),
    test_code,
    )  for i in range(len(test_y))
]

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("features", ArrayType(IntegerType()), True),
    StructField("label", IntegerType(), True),
    StructField("code", IntegerType(), True),
])

In [ ]:
train_df = spark.createDataFrame(train_data, schema=schema)
test_df = spark.createDataFrame(test_data, schema=schema)

In [ ]:
train_df.show()

In [ ]:
test_df.show()

In [ ]:
df_concat = train_df.union(test_df)
df_concat.show(5)

In [ ]:
dense_vector_to_list = udf(lambda v: list(v), ArrayType(FloatType()))

In [ ]:
train_df = train_df.withColumn("features", dense_vector_to_list(train_df["features"]))
test_df = test_df.withColumn("features", dense_vector_to_list(test_df["features"]))

In [ ]:
train_features = np.array(train_df.select("features").collect(),dtype=np.float32)
train_labels = np.array(train_df.select("label").collect(),dtype=np.int32)
test_features = np.array(train_df.select("features").collect(),dtype=np.float32)
test_labels = np.array(train_df.select("label").collect(),dtype=np.int32)

In [ ]:
train_dataset= tf.data.Dataset.from_tensor_slices((train_features, train_labels))
train_dataset=train_dataset.shuffle(buffer_size=len(train_features)).batch(64)

## CNN_Model_One

In [ ]:
def cnn_model_one():
    
    #creating a sequential instance to add layers to model
    cnn_model = Sequential()
    
    #first convolutional layer
    cnn_model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
    #first pooling layer
    cnn_model.add(MaxPooling2D((2,2), strides = 2))
    
    #second convolutional layer
    cnn_model.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    #second pooling layer
    cnn_model.add(MaxPooling2D((2,2), strides = 2))
    
    #third convolutional layer
    cnn_model.add(Conv2D(128, (3,3), padding='same', activation = 'relu'))
    #third pooling layer
    cnn_model.add(MaxPooling2D((2,2), strides = 2))
    
    #Flatten and connect layers
    cnn_model.add(Flatten())
    cnn_model.add(Dense(128, activation = 'relu'))
    
    #Fully connected layer of 10 to refelct 10 labels and softmax activation
    cnn_model.add(Dense(10, activation = 'softmax'))
    
    return cnn_model

In [ ]:
cnn_model_one = cnn_model_one()

# set up model optimizer, loss function, and accuracy
#adam - adaptive moment estimation
cnn_model_one.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

cnn_model_one.summary()

In [ ]:
#training the model
t0_one=time.time()
train_model_one = cnn_model_one.fit(train_X, train_y, epochs=10, validation_split=0.33)
print("Training time:", time.time()-t0_one)

In [ ]:
#test the model
loss, acc = cnn_model_one.evaluate(test_X, test_y, verbose = 1)
print('\nTest accuracy: ', acc)
print('\nTest loss: ', loss)

In [ ]:
#plotting the accuracy
plt.plot(train_model_one.history['sparse_categorical_accuracy'])
plt.plot(train_model_one.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy - CNN One')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the loss
plt.plot(train_model_one.history['loss'])
plt.plot(train_model_one.history['val_loss'])
plt.title('Model Loss - CNN One')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## CNN_Model_One - Epoch Increase

In [ ]:
#training the model
t0_one_epoch.time()
train_model_one_epoch = cnn_model_one.fit(train_X, train_y, epochs=30, validation_split=0.33)
print("Training time:", time.time()-t0_one_epoch)

In [ ]:
#test the model
loss, acc = cnn_model_one.evaluate(test_X, test_y, verbose = 1)
print('\nTest accuracy: ', acc)
print('\nTest loss: ', loss)

In [ ]:
#plotting the accuracy
fig, ax1 = plt.subplots()

ax1.plot(train_model_one_epoch.history['sparse_categorical_accuracy'], label='Train')
ax1.set_xlabel('epoch')
ax1.set_ylabel('Train')
ax1.tick_params(axis='y')

ax2 = ax1.twinx()

ax2.plot(train_model_one_epoch.history['val_sparse_categorical_accuracy'], label='Validation', color='orange')
ax2.set_ylabel('Validation')
ax2.tick_params(axis='y')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Model Accuracy - CNN One Epoch Increase')
plt.show()

In [ ]:
# plotting the loss
fig, ax1 = plt.subplots()

ax1.plot(train_model_one_epoch.history['loss'], label='Train')
ax1.set_xlabel('epoch')
ax1.set_ylabel('Train')
ax1.tick_params(axis='y')

ax2 = ax1.twinx()

ax2.plot(train_model_one_epoch.history['val_loss'], label='Validation', color='orange')
ax2.set_ylabel('Validation')
ax2.tick_params(axis='y')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Model Loss - CNN One Epoch Increase')
plt.show()

## CNN_Model_Two

In [ ]:
def cnn_model_two():
    
    #creating a sequential instance to add layers to model
    cnn_model_two = Sequential()
    
    #first convolutional layer
    cnn_model_two.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
    #first pooling layer
    cnn_model_two.add(MaxPooling2D((2,2), strides = 2))
    
    #second convolutional layer
    cnn_model_two.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    #second pooling layer
    cnn_model_two.add(MaxPooling2D((2,2), strides = 2))
    
    #third convolutional layer
    cnn_model_two.add(Conv2D(128, (3,3), padding='same', activation = 'relu'))
    #third pooling layer
    cnn_model_two.add(MaxPooling2D((2,2), strides = 2))
    
    #Flatten and connect layers
    cnn_model_two.add(Flatten())
    cnn_model_two.add(Dense(128, activation = 'relu'))
    cnn_model_two.add(Dense(128, activation = 'relu'))
    cnn_model_two.add(Dense(128, activation = 'relu'))
    cnn_model_two.add(Dense(128, activation = 'relu'))
    cnn_model_two.add(Dense(128, activation = 'relu'))
    cnn_model_two.add(Dense(128, activation = 'relu'))
    
    #Fully connected layer of 10 to refelct 10 labels and softmax activation
    cnn_model_two.add(Dense(10, activation = 'softmax'))
    
    return cnn_model_two

In [ ]:
cnn_model_two = cnn_model_two()

# set up model optimizer, loss function, and accuracy
#adam - adaptive moment estimation
cnn_model_two.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

cnn_model_two.summary()

In [ ]:
#training the model
t0_two=time.time()
train_model_two = cnn_model_two.fit(train_X, train_y, epochs=10, validation_split=0.33)
print("Training time:", time.time()-t0_two)

In [ ]:
#test the model
loss, acc = cnn_model_two.evaluate(test_X, test_y, verbose = 1)
print('\nTest accuracy: ', acc)
print('\nTest loss: ', loss)

In [ ]:
#plotting the accuracy
plt.plot(train_model_two.history['sparse_categorical_accuracy'])
plt.plot(train_model_two.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy - CNN Two')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the loss
plt.plot(train_model_two.history['loss'])
plt.plot(train_model_two.history['val_loss'])
plt.title('Model Loss - CNN Two')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## CNN_MODEL_THREE

In [ ]:
def cnn_model_three():
    
    #creating a sequential instance to add layers to model
    cnn_model_three = Sequential()
    
    #first convolutional layer
    cnn_model_three.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
    #first pooling layer
    cnn_model_three.add(MaxPooling2D((2,2), strides = 2))
    #add dropout
    model.add(Dropout(0.3)) 
    
    #second convolutional layer
    cnn_model_three.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    #second pooling layer
    cnn_model_three.add(MaxPooling2D((2,2), strides = 2))
    #add dropout
    model.add(Dropout(0.3)) 
    
    #third convolutional layer
    cnn_model_three.add(Conv2D(128, (3,3), padding='same', activation = 'relu'))
    #third pooling layer
    cnn_model_three.add(MaxPooling2D((2,2), strides = 2))
    #add dropout
    model.add(Dropout(0.3)) 
    
    #Flatten and connect layers
    cnn_model_three.add(Flatten())
    cnn_model_three.add(Dense(128, activation = 'relu'))
    cnn_model_three.add(Dense(128, activation = 'relu'))
    cnn_model_three.add(Dense(128, activation = 'relu'))
    cnn_model_three.add(Dense(128, activation = 'relu'))
    cnn_model_three.add(Dense(128, activation = 'relu'))
    cnn_model_three.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.3)) 
    
    #Fully connected layer of 10 to refelct 10 labels and softmax activation
    cnn_model_three.add(Dense(10, activation = 'softmax'))
    
    return cnn_model_three

In [ ]:
cnn_model_three = cnn_model_three()

# set up model optimizer, loss function, and accuracy
#adam - adaptive moment estimation
cnn_model_three.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

cnn_model_three.summary()

In [ ]:
#training the model
t0_three = time.time()
train_model_three = cnn_model_three.fit(train_X, train_y, epochs=10, validation_split=0.33)
print("Training time:", time.time()-t0_three)

In [ ]:
#test the model
loss, acc = cnn_model_three.evaluate(test_X, test_y, verbose = 1)
print('\nTest accuracy: ', acc)
print('\nTest loss: ', loss)

In [ ]:
#plotting the accuracy
plt.plot(train_model_three.history['sparse_categorical_accuracy'])
plt.plot(train_model_three.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy - CNN Three')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the loss
plt.plot(train_model_three.history['loss'])
plt.plot(train_model_three.history['val_loss'])
plt.title('Model Loss - CNN Three')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## CNN_Model_Four

In [ ]:
def cnn_model_four():
    
    #creating a sequential instance to add layers to model
    cnn_model_four = Sequential()
    
    #first convolutional layer
    cnn_model_four.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
    #add batch normalization
    cnn_model_four.add(BatchNormalization())
    #first pooling layer
    cnn_model_four.add(MaxPooling2D((2,2), strides = 2))
    
    
    #second convolutional layer
    cnn_model_four.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    #add batch normalization
    cnn_model_four.add(BatchNormalization())
    #second pooling layer
    cnn_model_four.add(MaxPooling2D((2,2), strides = 2))

    #third convolutional layer
    cnn_model_four.add(Conv2D(128, (3,3), padding='same', activation = 'relu'))
    #add batch normalization
    cnn_model_four.add(BatchNormalization())
    #third pooling layer
    cnn_model_four.add(MaxPooling2D((2,2), strides = 2))

    #Flatten and connect layers
    cnn_model_four.add(Flatten())
    cnn_model_four.add(Dense(128, activation = 'relu'))
    cnn_model_four.add(Dense(128, activation = 'relu'))
    cnn_model_four.add(Dense(128, activation = 'relu'))
    cnn_model_four.add(Dense(128, activation = 'relu'))
    cnn_model_four.add(Dense(128, activation = 'relu'))
    cnn_model_four.add(Dense(128, activation = 'relu'))
    cnn_model_four.add(Dropout(0.5))
    
    #Fully connected layer of 10 to refelct 10 labels and softmax activation
    cnn_model_four.add(Dense(10, activation = 'softmax'))
    
    return cnn_model_four

In [ ]:
cnn_model_four = cnn_model_four()

# set up model optimizer, loss function, and accuracy
#adam - adaptive moment estimation
cnn_model_four.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

cnn_model_four.summary()

In [ ]:
#training the model
t0_four = time.time()
train_model_four = cnn_model_four.fit(train_X, train_y, epochs=10, validation_split=0.33)
print("Training time:", time.time()-t0_four)

In [ ]:
#test the model
loss, acc = cnn_model_four.evaluate(test_X, test_y, verbose = 1)
print('\nTest accuracy: ', acc)
print('\nTest loss: ', loss)

In [ ]:
#plotting the accuracy
plt.plot(train_model_four.history['sparse_categorical_accuracy'])
plt.plot(train_model_four.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy - CNN Four')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#plotting the loss
plt.plot(train_model_four.history['loss'])
plt.plot(train_model_four.history['val_loss'])
plt.title('Model Loss - CNN Four')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## CNN_Model_Five

In [ ]:
def cnn_model_four():
    
    #creating a sequential instance to add layers to model
    cnn_model_four = Sequential()
    
    #first convolutional layer
    cnn_model_four.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
    cnn_model_four.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
    cnn_model_four.add(MaxPooling2D((2,2), strides = 2))
    cnn_model_four.add(Dropout(0.2))
    cnn_model_four.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    cnn_model_four.add(Conv2D(64, (3,3), padding='same', activation = 'relu'))
    cnn_model_four.add(BatchNormalization())
    cnn_model_four.add(MaxPooling2D((2,2), strides = 2))
    cnn_model_four.add(Dropout(0.3))

    #Flatten and connect layers
    cnn_model_four.add(Flatten())
    cnn_model_four.add(Dense(256, activation = 'relu'))
    cnn_model_four.add(BatchNormalization())
    cnn_model_four.add(Dropout(0.5))
    cnn_model_four.add(Dropout(0.7))

    
    #Fully connected layer of 10 to refelct 10 labels and softmax activation
    cnn_model_four.add(Dense(10, activation = 'softmax'))
    
    return cnn_model_four

In [ ]:
cnn_model_four = cnn_model_four()

# set up model optimizer, loss function, and accuracy
#adam - adaptive moment estimation
cnn_model_four.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

cnn_model_four.summary()

In [ ]:
# training the model
t0_four = time.time()
train_model_four = cnn_model_four.fit(train_X, train_y, epochs=10, validation_split=0.33)
print("Training time:", time.time()-t0_four)

In [ ]:
#test the model
loss, acc = cnn_model_four.evaluate(test_X, test_y, verbose = 1)
print('\nTest accuracy: ', acc)
print('\nTest loss: ', loss)